[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming Methods

## Convex Optimization with Disciplined Convex Programming (DCP) - Bounding Circle

Calculating the minimum area circle bounding a set of points.

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.003 | 07/01/2025 | Royi Avital | Added a question on single parameter solution, Drawing center      |
| 1.0.002 | 03/12/2024 | Royi Avital | Added a question to compare solvers                                |
| 1.0.001 | 03/03/2024 | Royi Avital | Added a question to compare to the mean                            |
| 1.0.000 | 09/02/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0011DCPBoundingCircle.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Optimization
import cvxpy as cp

# Miscellaneous
import os
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
from matplotlib.colors import LogNorm, Normalize, PowerNorm
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

## MathJaX Macros

Adding _quality of life_ macros.

$$
\newcommand{\MyParen}[1]{\left( #1 \right)}
\newcommand{\MyBrack}[1]{\left\lbrack #1 \right\rbrack}
\newcommand{\MyBrace}[1]{\left\lbrace #1 \right\rbrace}
\newcommand{\MyMat}[1]{\begin{bmatrix} #1 \end{bmatrix}}
\newcommand{\MyNorm}[2]{{\left\| #1 \right\|}_{#2}}
\newcommand{\MyAbs}[1]{\left| #1 \right|}
\newcommand{\MyNormTwo}[1]{\MyNorm{#1}{2}}
\newcommand{\MyCeil}[1]{\lceil #1 \rceil}
\newcommand{\MyInProd}[2]{\langle #1, #2 \rangle}
\newcommand{\MyUndBrace}[2]{\underset{#2}{\underbrace{#1}}}
\newcommand{\RR}[1]{\mathbb{R}^{#1}}
\newcommand{\InR}[1]{\in \mathbb{R}^{#1}}
\newcommand{\InC}[1]{\in \mathbb{C}^{#1}}
\newcommand{\BS}[1]{\boldsymbol{#1}}
\newcommand{\MyClr}[2]{{\color{#1}{#2}}}
\newcommand{\MyQuad}[2]{ {#1}^{T} #2 #1 }
$$

In [ ]:
# Configuration
%matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions

In [ ]:
# Parameters

# Data
numSamples  = 20
dataDim     = 2

# Visualization
tuAxisLim = (-2, 2)

## Disciplined Convex Programming (DCP)

The DCP is about formulating an optimization problem using DCP atoms and a set of composition rules.  
By obeying those rules the problem is guaranteed to be _convex_.  
The DCP parser then reformulate it a standard form which a solver can solve.

See [Stanford - Disciplined Convex Programming](https://fenchel.stanford.edu/home) for more information.

This notebook will solve a known problem using Python's [CVXPY](https://www.cvxpy.org).


* <font color='brown'>(**#**)</font> The first DCP package was created by Michael Grant for MATLAB - [CVX](http://cvxr.com).

## Minimum Area Bounding Circle

Given a set of points $\MyBrace{ \BS{x}_{i} \InR{2} }_{i = 1}^{N}$, find the circle with the minimum area which bounds all points.

* <font color='brown'>(**#**)</font> This problem is also known, in some formulations, as [Chebyshev Center](https://en.wikipedia.org/wiki/Chebyshev_center).

## Generate Data

Set of points in $\RR{2}$.

In [ ]:
# Generate / Load the Data

mX = 2 * (np.random.rand(numSamples, dataDim) - 0.5) #<! Each row is a data sample

In [ ]:
# Display Data

hF, hA = plt.subplots(figsize = (16, 8))
hA.scatter(mX[:, 0], mX[:, 1], s = 50, label = 'Samples')
hA.set_aspect('equal')
hA.set_xlim(tuAxisLim)
hA.set_ylim(tuAxisLim)
hA.set_xlabel(r'$x_1$')
hA.set_ylabel(r'$x_2$')
hA.set_title('Data Samples')

hA.legend();


### Question 001

Formulate a _convex_ problem to build the circle.  

**Hints**:

1. The circle is composed of a center point $\BS{c} \InR{2}$ and a radius $r \InR{}$.
2. Every point must be within the areas of the circle.



### Solution 001

 - The concept is to minimize the radius, $r$, which minimizes the area.
 - Each point must be at most with distance $r$ from the center.

 This yields:

$$
\begin{align}
\arg \min_{S} \quad & r \\
\text{subject to} \quad & \begin{aligned} 
\MyNormTwo{\BS{x}_{i} - \BS{c}} & \leq r \\
\end{aligned}
\end{align}
$$

---

In [ ]:
# Solving the Problem using CVXPY

vC          = cp.Variable(dataDim)
valRadius   = cp.Variable(1)

cpObjFun = cp.Minimize(valRadius) #<! Objective Function
cpConst = [cp.norm(mX[ii, :] - vC) <= valRadius for ii in range(numSamples)] #<! Constraint per each sample
oCvxPrb = cp.Problem(cpObjFun, cpConst)
oCvxPrb.solve(solver = cp.CLARABEL)

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

In [10]:
# Decode

vC = vC.value
valRadius = valRadius.value

In [ ]:
# Display Result

hF, hA = plt.subplots(figsize = (16, 8))
hA.scatter(mX[:, 0], mX[:, 1], s = 50, label = 'Samples')
patchCirc = Circle(vC, valRadius, color = 'r', lw = 2.5, fill = False, label = 'Bounding Circle')
hA.add_patch(patchCirc)
hA.scatter(vC[0], vC[1], s = 50, label = 'Center of Circle')
hA.set_aspect('equal')
hA.set_xlim(tuAxisLim)
hA.set_ylim(tuAxisLim)
hA.set_xlabel(r'$x_1$')
hA.set_ylabel(r'$x_2$')
hA.set_title('Data Samples')

hA.legend();


* <font color='blue'>(**!**)</font> Implement using a single parameter vector `vP`.   
  You may find `cp.vdot()` useful.
* <font color='blue'>(**!**)</font> Compare the result to the mean of the data. Explain the result.
* <font color='red'>(**?**)</font> How would you solve for the bounding square? Must we use `CVXPY` for that?  
  You may assume the square is aligned to the axis.
* <font color='green'>(**@**)</font> Implement, using `CVXPY` a solution for the bounding square.
* <font color='blue'>(**!**)</font> Try different solvers as in [CVXPY Solvers](https://www.cvxpy.org/tutorial/solvers). Compare Run Time.